In [ ]:
import MNIST_database as mnist
from matplotlib import pyplot as plt


#Choose the final size of your image dataset
size_final = 8

# data_zoom = mnist.MNISTData(size_initial=20, size_final=size_final, color_depth=5, flat=True)
data_zoom = mnist.MNISTData(size_initial=20, size_final=8, color_depth=8, flat=True)

x_train= data_zoom.x_train
y_train= data_zoom.y_train
x_test= data_zoom.x_test
x_test= data_zoom.y_test

ax = plt.subplot(1, 1 , 1)

plt.imshow(x_train[0].reshape(size_final,size_final), cmap='gray_r')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

In [ ]:
import tensorflow as tf
import qkeras as qk
from tensorflow.python.keras import Input
from tensorflow.keras.layers import Dense, Activation

# Define the input shape
# input_shape = (28, 28, 1)
input_shape_var = (x_train.shape[-1],)

# Create a sequential model
# model = tf.keras.Sequential()
# model = tf.keras.Sequential(input = input)
# model = tf.keras.Model(input = input)

input_layer  = Input(shape=input_shape_var, name='encoder_input')

# Add a QDense layer with 4 output units
# qk.qlayers.QDense(4, input_shape=input_shape))
encoder = qk.qlayers.QDense(4, input_shape=input_shape_var)(input_layer)
# Add a QActivation layer with relu activation
encoder = qk.qlayers.QActivation('relu', name='relu1')(encoder)

# Add a QDense layer with 3 output units
encoder = qk.qlayers.QDense(3)(encoder)
# Add a QActivation layer with relu activation
encoder = qk.qlayers.QActivation('relu', name='relu2')(encoder)


# Add a QDense layer with 2 output units
encoder = qk.qlayers.QDense(2)(encoder)
# # Add a QActivation layer with relu activation
# encoder = qk.qlayers.QActivation('relu')

# encoder = Dense(2,  activation='relu',
#                         name='encoder_output')(encoder)
# encoder_model = tf.keras.Model(encoder_input, encoder, name='encoder')

# Add a Dense layer with 2 output units
encoder_output = Dense(2, activation='relu', name='encoder_output')(encoder)
encoder_model = tf.keras.Model(input_layer, encoder_output)

# --------------------------------------------

# # Create a separate input for the decoder
# decoder_input = Input(shape=(2,), name='decoder_input')


# Add a QDense layer with 3 output units
decoder = qk.qlayers.QDense(3)(encoder_output)
# Add a QActivation layer with relu activation
decoder = qk.qlayers.QActivation('relu', name='relu3')(decoder)

# Add a QDense layer with 4 output units
decoder = qk.qlayers.QDense(4)(decoder)
# Add a QActivation layer with relu activation
decoder = qk.qlayers.QActivation('relu', name='relu4')(decoder)

# # Add a QDense layer with 4 output units
# decoder = qk.qlayers.QDense(input_shape[0])(decoder)
# # Add a QActivation layer with softmax activation
# decoder = qk.qlayers.QActivation('softmax', name='softmax')(decoder)

# # Add a QDense layer with 4 output units
# decoder_output = qk.qlayers.QDense(input_shape[0])(decoder)
# # Add a QActivation layer with softmax activation
# decoder_output = qk.qlayers.QActivation('softmax', name='softmax')(decoder_output)

# # Connect the encoder and decoder
# autoencoder = tf.keras.Model(inputs=[input_layer , decoder_input], outputs=decoder_output)
# # Compile the model
# autoencoder.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# # Print the model summary
# autoencoder.summary()




autoencoder = tf.keras.Model(
    input_layer, outputs=decoder, name='autoencoder')

# Compile the model
autoencoder.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Print the model summary
autoencoder.summary()


In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
import numpy as np
import tensorflow_model_optimization as tfmot

# Load your dataset here, e.g., x_train, y_train
# ...

# Define input shape
input_shape = (x_train.shape[-1],)

# Quantize the layers
def apply_quantization(layer):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)

# Encoder
encoder_input = Input(shape=input_shape, name='encoder_input')
encoder_layer1 = apply_quantization(Dense(128, activation='relu', name='encoder_layer1'))(encoder_input)
encoder_layer2 = apply_quantization(Dense(64, activation='relu', name='encoder_layer2'))(encoder_layer1)
encoder_layer3 = apply_quantization(Dense(32, activation='relu', name='encoder_layer3'))(encoder_layer2)

# Decoder
decoder_input = Input(shape=(32,), name='decoder_input')
decoder_concat = Concatenate()([encoder_layer3, decoder_input])
decoder_layer1 = apply_quantization(Dense(64, activation='relu', name='decoder_layer1'))(decoder_concat)
decoder_layer2 = apply_quantization(Dense(128, activation='relu', name='decoder_layer2'))(decoder_layer1)
decoder_output = Dense(y_train.shape[-1], activation='linear', name='decoder_output')(decoder_layer2)

# Create the model
model = Model(inputs=[encoder_input, decoder_input], outputs=decoder_output)

# Apply quantization
model = tfmot.quantization.keras.quantize_apply(model)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
# Assume the decoder's input is the same as the encoder's output during training
model.fit([x_train, np.zeros((x_train.shape[0], 32))], y_train, epochs=10, batch_size=32)


1875/1875 [==============================] - 12s 5ms/step - loss: 0.0181
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0089
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0071
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0064
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0057
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0052
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0048
Epoch 8/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0045
Epoch 9/10
1875/1875 [==============================] - 8s 5ms/step - loss: 0.0042
Epoch 10/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0039


In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import numpy as np
import tensorflow_model_optimization as tfmot

# Load your dataset here, e.g., x_train, y_train
# ...

# Define input shape
input_shape = (x_train.shape[-1],)

# Quantize the layers
def apply_quantization(layer):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)

# Encoder
encoder_input = Input(shape=input_shape, name='encoder_input')
encoder_layer1 = apply_quantization(Dense(128, activation='relu', name='encoder_layer1'))(encoder_input)
encoder_layer2 = apply_quantization(Dense(64, activation='relu', name='encoder_layer2'))(encoder_layer1)
encoder_output = apply_quantization(Dense(32, activation='relu', name='encoder_layer3'))(encoder_layer2)

# Decoder
decoder_layer4 = apply_quantization(Dense(64, activation='relu', name='decoder_layer4'))(encoder_output)
decoder_layer5 = apply_quantization(Dense(128, activation='relu', name='decoder_layer5'))(decoder_layer4)
decoder_output = Dense(y_train.shape[-1], activation='linear', name='decoder_output')(decoder_layer5)

# Create the model
model = Model(inputs=encoder_input, outputs=decoder_output)

# Apply quantization
model = tfmot.quantization.keras.quantize_apply(model)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
1875/1875 [==============================] - 10s 4ms/step - loss: 0.0182
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0087
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0071
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0062
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0055
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0050
Epoch 7/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0046
Epoch 8/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0043
Epoch 9/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0040
Epoch 10/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0038


In [28]:
# Get the weights and biases of the loaded model
for layer in model.layers:
    weights = layer.get_weights()
    print(f'Layer {layer.name}:')
    
    if len(weights) == 2:
        w, b = weights
        print(f'  Weights:\n{w}')
        print(f'  Biases:\n{b}')
    else:
        print('  No weights and biases.')

Layer encoder_input:
  No weights and biases.
Layer quantize_layer_2:
  No weights and biases.
Layer quant_encoder_layer1:
  No weights and biases.
Layer quant_encoder_layer2:
  No weights and biases.
Layer quant_encoder_layer3:
  No weights and biases.
Layer quant_decoder_layer4:
  No weights and biases.
Layer quant_decoder_layer5:
  No weights and biases.
Layer decoder_output:
  Weights:
[[-0.20701064 -0.17265531  0.07631456 ...  0.0121364   0.17497708
   0.05646818]
 [ 0.21986058 -0.08627374 -0.05961151 ... -0.02268347  0.06751335
  -0.07447217]
 [-0.09029378  0.14219518  0.12566659 ... -0.12262254 -0.12713194
  -0.13434418]
 ...
 [ 0.11256388 -0.17482634 -0.05767298 ...  0.17343871  0.0959226
   0.24154058]
 [-0.05519816 -0.08447433 -0.1041726  ...  0.02123571  0.01905223
  -0.07375693]
 [-0.02484782  0.10674526  0.08047733 ...  0.08869215  0.10797627
   0.09085944]]
  Biases:
[0.06200311 0.09120873 0.06115634 0.08244468 0.04640969 0.07651157
 0.03705138 0.03694129 0.0598756  0.102

In [30]:
# Save the model to a file
model.save('my_model.h5')

In [31]:
# Load the model from the saved file
loaded_model = tf.keras.models.load_model('my_model.h5')

ValueError: Unknown layer: 'QuantizeLayer'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
# Get the weights and biases of the loaded model
for layer in loaded_model.layers:
    weights = layer.get_weights()
    print(f'Layer {layer.name}:')
    
    if len(weights) == 2:
        w, b = weights
        print(f'  Weights:\n{w}')
        print(f'  Biases:\n{b}')
    else:
        print('  No weights and biases.')

In [ ]:
# import tensorflow as tf
# import qkeras as qk
# from tensorflow.python.keras import Input

# # Define the input shape
# # input_shape = (28, 28, 1)
# input_shape = (x_train.shape[-1],)

# # Create a sequential model
# model = tf.keras.Sequential()
# # model = tf.keras.Sequential(input = input)
# # model = tf.keras.Model(input = input)

# # input = Input(shape=input_shape, name='encoder_input')
# # model.add(input)

# # Add a QDense layer with 4 output units
# # model.add(qk.qlayers.QDense(4, input_shape=input_shape))
# model.add(qk.qlayers.QDense(4, input_shape=input_shape))
# # Add a QActivation layer with relu activation
# model.add(qk.qlayers.QActivation('relu'))

# # Add a QDense layer with 3 output units
# model.add(qk.qlayers.QDense(3))
# # Add a QActivation layer with relu activation
# model.add(qk.qlayers.QActivation('relu'))


# # Add a QDense layer with 2 output units
# model.add(qk.qlayers.QDense(2))
# # Add a QActivation layer with relu activation
# model.add(qk.qlayers.QActivation('relu'))

# # Add a QDense layer with 3 output units
# model.add(qk.qlayers.QDense(3))
# # Add a QActivation layer with relu activation
# model.add(qk.qlayers.QActivation('relu'))

# # Add a QDense layer with 4 output units
# model.add(qk.qlayers.QDense(4))
# # Add a QActivation layer with relu activation
# model.add(qk.qlayers.QActivation('relu'))

# # Add a QDense layer with 4 output units
# model.add(qk.qlayers.QDense(input_shape[0]))
# # Add a QActivation layer with softmax activation
# model.add(qk.qlayers.QActivation('softmax'))

# # Compile the model
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# # Print the model summary
# model.summary()


In [ ]:
input_shape

In [ ]:
x_train.shape

In [ ]:
Y_train.shape

In [ ]:
X_test.shape

In [ ]:
Y_test.shape

In [ ]:
# # Train the model
# model.fit(x_train, Y_train,
#           batch_size=128,
#           epochs=10,
#           validation_data=(X_test, Y_test))

# # Save the model
# model.save('my_model.h5')

# # Load the model
# loaded_model = tf.keras.models.load_model('my_model.h5', custom_objects={'QActivation': qk.qlayers.QActivation})

